# Ingest constructors.json file

## Pre-requisites
- Add data source and file date parameter widgets to the notebook and get it as a variable
- Run configuration notebook with folder paths
- Run common_functions notebook

In [0]:
dbutils.widgets.text("param_data_source", "")
var_data_source = dbutils.widgets.get("param_data_source")

In [0]:
dbutils.widgets.text("param_file_date", "")
var_file_date = dbutils.widgets.get("param_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Add ingestion date column to each data frame using current timestamp

### Step 1 - Read the JSON file using the Spark DataFrame Reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
constructors_schema = StructType(fields = [
    StructField("constructorId", IntegerType(), False),
    StructField("constructorRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("nationality", StringType(), False),
    StructField("url", StringType(), True)
])

In [0]:
constructors_df = spark.read \
    .schema(constructors_schema) \
    .json(f'{raw_folder_path}/{var_file_date}/constructors.json')

## Step 2 - Drop unwanted column

In [0]:
constructors_dropped_df = constructors_df.drop(constructors_df['url'])

## Step 3 - Rename columns and add ingestion date column

In [0]:
from pyspark.sql.functions import lit

In [0]:
constructors_renamed_df = constructors_dropped_df.withColumnRenamed("constructorId", "constructor_id") \
                                                .withColumnRenamed("constructorRef", "constructor_ref") \
                                                .withColumn("data_source", lit(var_data_source)) \
                                                .withColumn("file_date", lit(var_file_date))

In [0]:
constructors_final_df = add_ingestion_date(constructors_renamed_df)

## Step 4 - Write data to Data Lake as a managed table in Delta Lake format

In [0]:
constructors_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.constructors")

In [0]:
dbutils.notebook.exit("Success")